In [12]:
from sklearn.externals import joblib

rf = joblib.load( 'random_forest.pkl')

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

engine = create_engine('sqlite:///MyDB.sqlite3', echo=True)

data = pd.read_sql("select level, hot,num, price, area from item;", con=engine)
data = data[(data['level'] != '')]

data["area"] = data["area"].apply(lambda x: x.replace("[", "").replace("]", ""))
data["province"] = data["area"].apply(lambda x: x.split("·")[0])
city = list(set(data['province']))
data['province'] = data['province'].apply(lambda x:city.index(x))

train_data = data[['price', 'num','hot', 'province']]

from sklearn.preprocessing import scale
import numpy as np

train_data = scale(train_data)

y = data[['level']]

# from imblearn.over_sampling import  SMOTE

# train_data, y =  SMOTE().fit_sample(train_data, y)

x_train,x_test,y_train,y_test = train_test_split(train_data,y,test_size=0.2)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

2018-10-25 19:43:57,882 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-10-25 19:43:57,883 INFO sqlalchemy.engine.base.Engine ()
2018-10-25 19:43:57,884 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-10-25 19:43:57,885 INFO sqlalchemy.engine.base.Engine ()
2018-10-25 19:43:57,885 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("select level, hot,num, price, area from item;")
2018-10-25 19:43:57,891 INFO sqlalchemy.engine.base.Engine ()
2018-10-25 19:43:57,893 INFO sqlalchemy.engine.base.Engine select level, hot,num, price, area from item;
2018-10-25 19:43:57,894 INFO sqlalchemy.engine.base.Engine ()
(1662, 4) (416, 4) (1662, 1) (416, 1)


/home/miku/anaconda2/envs/sun/lib/python3.5/site-packages/ipykernel_launcher.py:22: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.


In [14]:
y_pred = rf.predict(x_test)
score = rf.score(x_test,y_test)
print(score)

0.6802884615384616


In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

        3A景区       0.54      0.58      0.55        92
        4A景区       0.83      0.70      0.76       267
        5A景区       0.46      0.74      0.57        57

   micro avg       0.68      0.68      0.68       416
   macro avg       0.61      0.67      0.63       416
weighted avg       0.72      0.68      0.69       416

